In [43]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_score
import warnings

from datetime import datetime, timedelta


warnings.filterwarnings("ignore", category=FutureWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.datasets import load_sample_image
import matplotlib.pyplot as plt


from sklearn.cluster import KMeans, DBSCAN
import os


## Import dataset

In [44]:
path = "./data/01aJourneyDataExtract10Jan16-23Jan16.csv"
dataset = pd.read_csv(path)
dataset.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50754225,240,11834,10/01/2016 00:04,383.0,"Frith Street, Soho",10/01/2016 00:00,18,"Drury Lane, Covent Garden"
1,50754226,300,9648,10/01/2016 00:05,719.0,"Victoria Park Road, Hackney Central",10/01/2016 00:00,479,"Pott Street, Bethnal Green"
2,50754227,1200,10689,10/01/2016 00:20,272.0,"Baylis Road, Waterloo",10/01/2016 00:00,425,"Harrington Square 2, Camden Town"
3,50754228,780,8593,10/01/2016 00:14,471.0,"Hewison Street, Old Ford",10/01/2016 00:01,487,"Canton Street, Poplar"
4,50754229,600,8619,10/01/2016 00:11,399.0,"Brick Lane Market, Shoreditch",10/01/2016 00:01,501,"Cephas Street, Bethnal Green"


In [45]:
# df = dataset
# df[['End Date', 'Start Date']] = dataset[['End Date', 'Start Date']].apply(pd.to_datetime, dayfirst=True).apply(lambda x: x.dt.time)
# df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50754225,240,11834,00:04:00,383.0,"Frith Street, Soho",00:00:00,18,"Drury Lane, Covent Garden"
1,50754226,300,9648,00:05:00,719.0,"Victoria Park Road, Hackney Central",00:00:00,479,"Pott Street, Bethnal Green"
2,50754227,1200,10689,00:20:00,272.0,"Baylis Road, Waterloo",00:00:00,425,"Harrington Square 2, Camden Town"
3,50754228,780,8593,00:14:00,471.0,"Hewison Street, Old Ford",00:01:00,487,"Canton Street, Poplar"
4,50754229,600,8619,00:11:00,399.0,"Brick Lane Market, Shoreditch",00:01:00,501,"Cephas Street, Bethnal Green"


In [46]:
# morning = "09:00:00"
# evening = "17:00:00"
# morning_time = datetime.strptime(morning, "%H:%M:%S").time()
# evening_time = datetime.strptime(evening, "%H:%M:%S").time()
# print(morning_time, evening_time)

09:00:00 17:00:00


## Morning hours

In [47]:
# # Calcul de morning_start et morning_end
# morning_start = (datetime.combine(datetime.today(), morning_time) - timedelta(hours=1)).time()
# morning_end = (datetime.combine(datetime.today(), morning_time) + timedelta(hours=1)).time()

# df_morning = df[(df['Start Date'] >= morning_start) & (df['Start Date'] <= morning_end)]
# df_morning = df_morning.reset_index(drop=True)
# df_morning.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50755147,1140,740,08:19:00,308.0,"Long Lane , Bermondsey",08:00:00,546,"New Fetter Lane, Holborn"
1,50755148,360,9884,08:06:00,114.0,"Park Road (Baker Street), The Regent's Park",08:00:00,540,"Albany Street, The Regent's Park"
2,50755149,300,10063,08:06:00,320.0,"Queen Mother Sports Centre, Victoria",08:01:00,74,"Vauxhall Cross, Vauxhall"
3,50755150,240,10010,08:06:00,638.0,"Falcon Road, Clapham Junction",08:02:00,701,"Vicarage Crescent, Battersea"
4,50755151,480,8468,08:10:00,247.0,"St. John's Wood Church, The Regent's Park",08:02:00,535,"Gloucester Avenue, Camden Town"


In [48]:
# # Elimination colones superflues
# df_morning = df_morning.drop(["Duration","Rental Id", "End Date","EndStation Id", "EndStation Name"], axis=1)
# df_morning.head()

,Bike Id,Start Date,StartStation Id,StartStation Name
0,740,08:00:00,546,"New Fetter Lane, Holborn"
1,9884,08:00:00,540,"Albany Street, The Regent's Park"
2,10063,08:01:00,74,"Vauxhall Cross, Vauxhall"
3,10010,08:02:00,701,"Vicarage Crescent, Battersea"
4,8468,08:02:00,535,"Gloucester Avenue, Camden Town"


In [49]:
# df_morning.columns

Index(['Bike Id', 'Start Date', 'StartStation Id', 'StartStation Name'], dtype='object')

## Evening hours

In [50]:
# # Calcul de evening_start et evening_end
# evening_start = (datetime.combine(datetime.today(), evening_time) - timedelta(hours=1)).time()
# evening_end = (datetime.combine(datetime.today(), evening_time) + timedelta(hours=1)).time()

# df_evening = df[(df['Start Date'] >= evening_start) & (df['Start Date'] <= evening_end)]
# df_evening = df_evening.reset_index(drop=True)
# df_evening.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50764866,900,9305,16:15:00,116.0,"Little Argyll Street, West End",16:00:00,114,"Park Road (Baker Street), The Regent's Park"
1,50764867,720,10490,16:12:00,48.0,"Godliman Street, St. Paul's",16:00:00,123,"St. John Street, Finsbury"
2,50764868,840,2778,16:14:00,163.0,"Sloane Avenue, Knightsbridge",16:00:00,376,"Millbank Tower, Pimlico"
3,50764869,2760,8823,16:46:00,406.0,"Speakers' Corner 2, Hyde Park",16:00:00,432,"Exhibition Road Museums, South Kensington"
4,50764870,2520,5528,16:42:00,682.0,"Crisp Road, Hammersmith",16:00:00,681,"Bishop's Avenue, Fulham"


In [51]:
# Elimination colones superflues
df_evening = df_evening.drop(["Duration","Rental Id", "End Date","EndStation Id", "EndStation Name"], axis=1)
df_evening.head()

,Bike Id,Start Date,StartStation Id,StartStation Name
0,9305,16:00:00,114,"Park Road (Baker Street), The Regent's Park"
1,10490,16:00:00,123,"St. John Street, Finsbury"
2,2778,16:00:00,376,"Millbank Tower, Pimlico"
3,8823,16:00:00,432,"Exhibition Road Museums, South Kensington"
4,5528,16:00:00,681,"Bishop's Avenue, Fulham"


In [52]:
print(df_morning.dtypes)

Bike Id               int64
Start Date           object
StartStation Id       int64
StartStation Name    object
dtype: object


# Machine Learning
## Morning

In [53]:
# Echantillon pour mise en place
df_morning_sample = df_morning.sample(n=10000, random_state=4)
df_morning_sample.head()

,Bike Id,Start Date,StartStation Id,StartStation Name
26194,6,08:44:00,229,"Whitehall Place, Strand"
3200,8682,08:46:00,105,"Westbourne Grove, Bayswater"
8841,7722,08:46:00,95,"Aldersgate Street, Barbican"
36646,13263,08:05:00,548,"Westminster Bridge Road, Elephant & Castle"
5552,3573,09:45:00,540,"Albany Street, The Regent's Park"
